# YOUR PROJECT TITLE

Imports and set magics:

In [8]:
import numpy as np
from scipy import optimize
import sympy as sm
from scipy import linalg


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import SolowModelClass
model = SolowModelClass()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

The "base-line" model that we will look at, will be an extended solow model, the solow model with human capital. 

In the solow model with human capital, there is an inclusion of human capital in the production function and accumulation of human capital by consumers. * 
 

The solow model shows ....

This project will look at the variation with human capital and how the implemtation of human capital affects the growth. Firstly it will be assumed that the human capitala is equal to the countrys population and later we will change this by making the human capital a part of the total population. Finally, we wil see how the parameters $ \alpha$ , $ \phi$ and $\delta$ will affect the growth




. * "Introducing Advanced Macroeconomics, by H.J.Whitta-Jacobsen and P.B.Sørensen"

The Solow-model with Human capital is given as below, by the book "Introducing Advanced Macroeconomics, by H.J.Whitta- Jacobsen and P.B.Sørensen"


$$
Y_{t} = K_{t}^{\alpha} H_{t}^{\phi} (A_{t} L_{t})^{1-\alpha -\phi}
$$

$$
r_{t} = \alpha \left( \frac{K_{t}}{A_{t} L_{t}} \right)^{\alpha -1} \left(\frac{H_{t}}{A_{t} L_{t}}\right)^{\phi}
$$

$$
w{_t} = ( 1 - \alpha) \left( \frac{K_{t}}{A_{t} L_{t}} \right)^{\alpha} \left(\frac{H_{t}}{A_{t} L_{t}}\right)^{\phi} A_{t}
$$

$$
K_{t+1} - K_{t} = s_{K} Y_{t} - \delta K_{t}
$$

$$ 
H_{t+1} - H_{t} = s_{H} Y_{t} - \delta H_{t}
$$


$$
L_{t+1} = (1 + n) L_{t}
$$

$$
A_{t+1} = (1 + g) A_{t}
$$ 


$$
\tilde{k}_{t+1}= 
$$

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

1. Find Steady State
2. Compute the null-clines

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

1. Human Capital is a part of the populaiton - Interactive diagram?

$$
L_{t} = \lambda N_{t}
$$  
where $ 0< \lambda < 1 $

$$
N_{t+1} = (1 + n) N_{t} 
$$

2. How changes in alpha, phi and delta affect the growth - interactive diagram. 

# Conclusion

Add concise conclusion. 

In [14]:
production = model.ProductionFunction()



AttributeError: 'SolowModelClass' object has no attribute 'par'